In [1]:
import pandas as pd
import numpy as np

trades = pd.read_csv("../data/raw/historical_data.csv")
sentiment = pd.read_csv("../data/raw/fear_greed_index.csv")

# Convert Unix timestamp to datetime (UTC)
trades["trade_time"] = pd.to_datetime(trades["Timestamp"], unit="ms")

# Extract date for merging
trades["date"] = trades["trade_time"].dt.date

In [2]:
trades["trade_time"].min(), trades["trade_time"].max()

(Timestamp('2023-03-28 10:40:00'), Timestamp('2025-06-15 15:06:40'))

In [3]:
# Prepare sentiment date
sentiment["date"] = pd.to_datetime(sentiment["date"]).dt.date

sentiment_clean = sentiment[["date", "classification", "value"]].copy()

# Merge
master_df = trades.merge(
    sentiment_clean,
    on="date",
    how="left"
)

master_df.rename(columns={
    "classification": "market_sentiment",
    "value": "fear_greed_score"
}, inplace=True)

In [4]:
master_df.shape
master_df["market_sentiment"].isna().sum()
trades["trade_time"].head()

0   2024-10-27 03:33:20
1   2024-10-27 03:33:20
2   2024-10-27 03:33:20
3   2024-10-27 03:33:20
4   2024-10-27 03:33:20
Name: trade_time, dtype: datetime64[ns]

In [5]:
master_df = trades.merge(sentiment_clean, on="date", how="left")
master_df["classification"].isna().sum()


np.int64(26961)

In [6]:
master_df.loc[
    master_df["classification"].isna(),
    "date"
].value_counts().head()

date
2025-06-15    26961
Name: count, dtype: int64

In [8]:
# Save the master dataset for analysis
master_df.to_csv(
    "../outputs/master_trader_sentiment.csv",
    index=False
)
